# Language Modeling and Lyrics Generation
3 models are built for 3 most frequent genres to generate lyrics
A statistical model is built using bigram and the results are compared.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import os
import time

In [ ]:
#https://drive.google.com/file/d/1u-bq8vYG8BMT-szSoDiCpdkiO_8NNIS5/view?usp=sharing
!gdown --id "1u-bq8vYG8BMT-szSoDiCpdkiO_8NNIS5"

Downloading...
From: https://drive.google.com/uc?id=1u-bq8vYG8BMT-szSoDiCpdkiO_8NNIS5
To: /content/final.csv
675MB [00:03, 221MB/s]


In [ ]:
CLEANED_FILE_PATH = './final.csv'

In [ ]:
cleaned_data = pd.read_csv(CLEANED_FILE_PATH)
cleaned_data.head()

,Artist,Song,Genre,Language,Lyrics,cleanedlyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ...",it start with pain follow by hate fuel by the ...
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...,freedom alon again again alon patient wait by ...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v...",bite the hand that feed you lie to the voic in...
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...,you say you know just who i am but you cant im...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...,my heart is beat fast cant control these feel ...


In [ ]:
cleaned_data.dropna(inplace=True)

In [ ]:
genres_count = cleaned_data.groupby('Genre')['Genre'].count()

In [ ]:
top_3_genres = genres_count.nlargest(3).reset_index

In [ ]:
top_3_genres

<bound method Series.reset_index of Genre
Rock     121390
Pop      108693
Metal     20286
Name: Genre, dtype: int64>

# Top 3 genres are Rock Pop and Metal

## Metal Genre 
#### Run-time ~ 5-10 mins

In [ ]:
lyric_metal = cleaned_data['cleanedlyrics'].loc[(cleaned_data['Genre']=='Metal')&(cleaned_data['Language']=='en')]

In [ ]:
lyric_metal = lyric_metal.str.cat(sep=' ')

In [ ]:
print(f'Length of text: {len(lyric_metal)} characters')

Length of text: 19226774 characters


In [ ]:
lyric_metal[:50]

'a hundr day have made me old sinc the last time th'

In [ ]:
# exract an ordered vocabulary - this will be letters, some numbers etc. 
vocab = sorted(set(lyric_metal))
print(f'{len(vocab)} unique characters')

49 unique characters


In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(lyric_metal, 'UTF-8'))
all_ids

<tf.Tensor: shape=(19226774,), dtype=int64, numpy=array([ 3,  2, 10, ..., 17, 17, 10])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100
examples_per_epoch = len(lyric_metal)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'a hundr day have made me old sinc the last time that i saw your pretti face a thousand lie have made'
Target: b' hundr day have made me old sinc the last time that i saw your pretti face a thousand lie have made '


In [ ]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 51) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  13056     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  52275     
Total params: 4,003,635
Trainable params: 4,003,635
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
   )

In [ ]:
EPOCHS = 2

In [ ]:
history = model.fit(dataset, epochs=EPOCHS,callbacks=[checkpoint_callback],verbose=1)
print("training complete.")

Epoch 1/2
743/743 [==============================] - 146s 194ms/step - loss: 2.1817
Epoch 2/2
743/743 [==============================] - 148s 198ms/step - loss: 1.2630
training complete.


In [ ]:
def generate_one_step(model, inputs, states=None, temperature = 1):
    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = ids_from_chars(['', '[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['dance'])
result = [next_char]

for n in range(1000):
    next_char, states = generate_one_step(model, next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor([b'dancen sleep turn away from hi lyfitim ankow joy the show down now i tell thi night is fruzin begrie glow on i didnt need a toot in me not hold our hope it hurt feel enough and if is let get realli just a littl bite on cold street at deceas and pain they step malestim unfold it life is done miceri chanc but they catch me with it get betray weak the fresh is in faith is thi cattl secreci two behold dri it to desarv repent men but baggag te a tool be after wrong their ingani man crawl overdustain rminesion live lit trembebl onc age we will end over anoth night destind manifest monstrvist drive by their knee and the moon these bite of the earth they seek assam no one spot hi keeph so far stop their land of beast romain pumpin poison rule cant will be frustrat a culsul depend come inscroudlin fall scorn and cunt as i came tri to pay you but still i am i what left to spread in mind what you do what i remeng i pay fade i rememb i am uni beyond that it hope i shall prey i seek im

## Rock Genre
#### Run-time ~ 15-20 mins

In [ ]:
lyric_rock = cleaned_data['cleanedlyrics'].loc[(cleaned_data['Genre']=='Rock')&(cleaned_data['Language']=='en')]

In [ ]:
lyric_rock

0         it start with pain follow by hate fuel by the ...
1         freedom alon again again alon patient wait by ...
2         bite the hand that feed you lie to the voic in...
3         you say you know just who i am but you cant im...
4         my heart is beat fast cant control these feel ...
                                ...                        
290086    your a shoot star that is what you are you bro...
290093    life live alon on top of a hill tonight the mo...
290103    too much inform for our mind to comprehend thi...
290104    a monster use to chase me use to jump from the...
290123    what all thi space junk these gem behind my ey...
Name: cleanedlyrics, Length: 107144, dtype: object

In [ ]:
lyric_rock= lyric_rock.str.cat(sep=' ')

In [ ]:
print(f'Length of text: {len(lyric_rock)} characters')

Length of text: 99948513 characters


In [ ]:
lyric_rock[:50]

'it start with pain follow by hate fuel by the endl'

In [ ]:
# exract an ordered vocabulary - this will be letters, some numbers etc. 
vocab = sorted(set(lyric_rock))
print(f'{len(vocab)} unique characters')

59 unique characters


In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(lyric_rock, 'UTF-8'))
all_ids

<tf.Tensor: shape=(99948513,), dtype=int64, numpy=array([11, 22,  2, ...,  7,  3, 14])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100
examples_per_epoch = len(lyric_rock)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'it start with pain follow by hate fuel by the endless question no one can answer a stain cover your '
Target: b't start with pain follow by hate fuel by the endless question no one can answer a stain cover your h'


In [ ]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 61) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  15616     
_________________________________________________________________
gru_1 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  62525     
Total params: 4,016,445
Trainable params: 4,016,445
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
   )

In [ ]:
EPOCHS = 2

In [ ]:
history = model.fit(dataset, epochs=EPOCHS,callbacks=[checkpoint_callback],verbose=1)
print("training complete.")

Epoch 1/2
3865/3865 [==============================] - 770s 199ms/step - loss: 1.5741
Epoch 2/2
3865/3865 [==============================] - 767s 198ms/step - loss: 1.1282
training complete.


In [ ]:
def generate_one_step(model, inputs, states=None, temperature = 1):
    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = ids_from_chars(['', '[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['dance'])
result = [next_char]

for n in range(1000):
    next_char, states = generate_one_step(model, next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor([b'dancer miranda is a fredyfin serjon vile in our mirror bound wouldnt leav a sound if your wake in america and we found those thing we said fransilli of wretch togeth with tear are fall tear like the road to the bee part of them didnt take my breath you dirti thing consequ your your last time time seem for all thi sexi grey caus your laid park right your too backy act nearli broken new bottl ill rearrang for the day well i can see what youd fell apart the dog is you that i cant stand that sympathis with the bamboo off the corn the pric instit of color maset of cussa in high hors run out of control when the afternoon scotlers disconcert wife and famili and you dont care caus when will where it went without it listen to my wall a littl bit close confid wont no i will stand as deep so ill take you down so madli tight the king is burn on the answer are keep bring me id be glad to be thi old rage babi im a babi straight for i throw some twenti filth on the countri in jamazingl t

## Pop Genre
#### Run-time ~ 25-30 mins

In [ ]:
lyric_pop = cleaned_data['cleanedlyrics'].loc[(cleaned_data['Genre']=='Pop')&(cleaned_data['Language']=='en')]

In [ ]:
lyric_pop = lyric_pop.str.cat(sep=' ')

In [ ]:
print(f'Length of text: {len(lyric_pop)} characters')

Length of text: 125333310 characters


In [ ]:
lyric_pop[:50]

'twentyf year and my life is still tri to get up th'

In [ ]:
# exract an ordered vocabulary - this will be letters, some numbers etc. 
vocab = sorted(set(lyric_pop))
print(f'{len(vocab)} unique characters')

67 unique characters


In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(lyric_pop, 'UTF-8'))
all_ids

<tf.Tensor: shape=(125333310,), dtype=int64, numpy=array([22, 25,  7, ..., 17, 15,  7])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100
examples_per_epoch = len(lyric_pop)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'twentyf year and my life is still tri to get up that great big hill of hope for a destin i realiz qu'
Target: b'wentyf year and my life is still tri to get up that great big hill of hope for a destin i realiz qui'


In [ ]:
# Batch size
BATCH_SIZE = 256

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(256, 100, 69) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  17664     
_________________________________________________________________
gru_3 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_3 (Dense)              multiple                  70725     
Total params: 4,026,693
Trainable params: 4,026,693
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
   )

In [ ]:
EPOCHS = 2

In [ ]:
history = model.fit(dataset, epochs=EPOCHS,callbacks=[checkpoint_callback],verbose=1)
print("training complete.")

Epoch 1/2
4847/4847 [==============================] - 963s 198ms/step - loss: 1.5145
Epoch 2/2
4847/4847 [==============================] - 960s 198ms/step - loss: 1.1126
training complete.


In [ ]:
def generate_one_step(model, inputs, states=None, temperature = 1):
    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = ids_from_chars(['', '[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/temperature
        # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['dance'])
result = [next_char]

for n in range(1000):
    next_char, states = generate_one_step(model, next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor([b'danced painki brokin and rocki ni chica head hod ass nigga you search for myself what cheas u you you wear my hair deep deep in my face and left from my pave my crime to my place and a jamman they hang me get real fast what you make me tri that i had a hacu i know the day what i can chang can i be with you more than and i give my dimnt up poliss and you can turn around and breakawatch a bodi on each other sign i wish i took a while suddenli im tire of not a mess chain so babi you turn around i tri to tear you over you you say come around he move darl you give me your love i can see your throli no matter what you do now be treat my leav you give your love to me thi right where is bright green me so what are we gonna do whatch it gonna choic hasit sugar yo to help her out a hit background i dream drip oh kbock the cold and you leav her meet need is a talk unaboda when you touch it move your face high lifer in need more fubutti all thi diamond in the acr no put thi love in th